In [1]:
from whispr import *
import os


In [14]:
# gene expression. 2.5ul rxns, 75% txtl
rxn_vol = 10
source_plate_type = '384PP_AQ_BP' 

os.getcwd()
folder = os.getcwd() + '/Experiments/221213_ATP_regen/'
sp_plasmids_file = folder + 'arpae_plasmids.xlsx'
sp_plasmids = pd.read_excel(sp_plasmids_file, index_col = 0)
sp_plasmids = sp_plasmids[~sp_plasmids['Well'].isna()]

layout_genex_file = folder + 'atp-regen-genex-pl.csv'
layout_genex = pd.read_csv(layout_genex_file, index_col = 0, dtype = str)

mt_genex_file = folder + 'atp-regen-genex-mt.csv'
mt_genex = pd.read_csv(mt_genex_file, index_col = 0, dtype = str).fillna(0)

checkInputs(sp_plasmids, mt_genex,source_plate_type)

vol_table_df = generateVolumeTable(mt_genex, sp_plasmids, rxn_vol = 0.25*rxn_vol, total_vol = rxn_vol, fill_with='water', multiRpW = False)

#specify rxn_vol (default = 2.5) and total_vol (default = 10) if you'd like to change the volume of each individual replicate or the total reaction volume
protocol_genex_df = writeProtocol(source_plate_type, vol_table_df, layout_genex,sp_plasmids, update_source_vol=folder+'221213_arpae_plasmids_updated.xlsx')


Volumes used from each well for this protocol:
{'1_A1': 3.675, '1_P1': 0.5, '1_P2': 1.15, '1_P3': 0.75, '1_P5': 0.05, '1_P8': 0.55, '1_P9': 0.4, '1_P15': 0.425}


/Users/ryancardiff/Documents/GitHub/WHISPR/whispr.py:99: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  vol_table_df = vol_table_df.append({'Label': str(row)}, ignore_index = True)
/Users/ryancardiff/Documents/GitHub/WHISPR/whispr.py:99: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  vol_table_df = vol_table_df.append({'Label': str(row)}, ignore_index = True)
/Users/ryancardiff/Documents/GitHub/WHISPR/whispr.py:99: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  vol_table_df = vol_table_df.append({'Label': str(row)}, ignore_index = True)
/Users/ryancardiff/Documents/GitHub/WHISPR/whispr.py:345: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  

In [15]:
protocol_genex_df = protocol_genex_df[0]

In [18]:
vol_table_df

,Label,water,biotin,Pyc,Fdh,P70_GFP,Mdh,Mdh_10x,ME2,TdcB,PLP,SHMT,ICL,MS,ACS,Chi6,ppk12
0,ppk_minus,0.8,0.25,0.575,0.375,NaN,0.025,NaN,NaN,0.275,0.2,NaN,NaN,NaN,NaN,NaN,0.0
1,ppk_plus,0.375,0.25,0.575,0.375,NaN,0.025,NaN,NaN,0.275,0.2,NaN,NaN,NaN,NaN,NaN,0.425
2,Std_121322,2.5,0.0,0.0,0.0,NaN,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0


In [17]:
# everything gets txtl

# get destination wells
for layout_id, layout in enumerate([layout_genex]):    
    # reads plate layout and assigns wells to each reaction (rxn_loc; dict)
    labels = pd.unique(np.concatenate(layout.values))
    rxn_loc = {}
    for l in labels:
        if type(l) is str:
            index = layout[layout.isin([l])].stack().index
            rxn_loc[l] = []
            for i in index:
                rxn_loc[l].append(str(i[0]) + str(i[1]))


txtl_well = 'M24'
for label, dwell in rxn_loc.items():
    for d in dwell:
        print(label)
        txtl_vol = vol_table_df[vol_table_df['Label'] == label].iloc[0,1:].sum()/0.25*0.75*1000 # add txtl to make 75%
        row = {'Source Plate Name':'Source[1]', 'Source Plate Type': source_plate_type, 'Source Well': txtl_well,
                                    'Destination Plate Name':'Destination[1]', 'Destination Well': d, 'Transfer Volume': txtl_vol}
        protocol_genex_df = protocol_genex_df.append(row, ignore_index = True)

protocol_genex_df.to_csv(folder+'protocol_genex.csv',index = False)
print('TXTL needed: '+str(protocol_genex_df[protocol_genex_df['Source Well'] == txtl_well]['Transfer Volume'].sum()/1000))
# carry out gene expression reaction

ppk_plus
ppk_minus
Std_121322
TXTL needed: 22.5


/var/folders/55/z2skprpx0m3_6k8fs3mcm_mm0000gn/T/ipykernel_73095/455720547.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  protocol_genex_df = protocol_genex_df.append(row, ignore_index = True)
/var/folders/55/z2skprpx0m3_6k8fs3mcm_mm0000gn/T/ipykernel_73095/455720547.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  protocol_genex_df = protocol_genex_df.append(row, ignore_index = True)
/var/folders/55/z2skprpx0m3_6k8fs3mcm_mm0000gn/T/ipykernel_73095/455720547.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  protocol_genex_df = protocol_genex_df.append(row, ignore_index = True)


In [37]:
# dilution. everything gets Tris, for 20x -> 57uL

##### doing it by hand if not too many reactions #####

# protocol_dilution_df = pd.DataFrame(columns=protocol_genex_df.columns)
# source_plate_type = '6RES_AQ_B' 
# tris_well = 'A1'
# tris_vol = 57*1000

# for dwell in protocol_genex_df['Destination Well'].unique():
#     row = {'Source Plate Name':'Source[1]', 'Source Plate Type': source_plate_type, 'Source Well': tris_well,
#                                 'Destination Plate Name':'Destination[1]', 'Destination Well': dwell, 'Transfer Volume': tris_vol}
#     protocol_dilution_df = protocol_dilution_df.append(row, ignore_index = True)

# protocol_dilution_df.to_csv('protocols/221024_arpae_dilutions.csv',index = False)

In [ ]:
# from source plate with buffer, mix into 96

# from source plate with genex, mix the diluted txtl

In [38]:
# biosynthesis. 25ul rxns, 2.5ul of diluted txtl

# buffer source plate
sp_buffers_file = folder + 'UPDATE_buffers_sp.xlsx'
sp_buffers= pd.read_excel(sp_buffers_file, index_col = 0)
sp_buffers = sp_buffers[~sp_buffers['Well'].isna()]

# HEPES reservoir (actually same plate as Tris)
sp_hepes = pd.DataFrame(columns=sp_buffers.columns)
sp_hepes = sp_hepes.append({'Label':'HEPES', 'Well':'A2,A3', 'Concentration':'', 'Volume':'2000,2000'}, ignore_index = True)

# gene expression source plate (diluted w/ Tris)
sp_genex = sp_from_layout(layout_genex, 60)

# group source plates
sp_types = ['384PP_AQ_BP','6RES_AQ_BP2','384PP_AQ_BP'] # triple check this
sps = [sp_buffers,sp_hepes,sp_genex]

# get mixing table
mt_biosyn_file = folder + 'atp-regen-buffers-mt.csv'
mt_biosyn = pd.read_csv(mt_biosyn_file, index_col = 0, dtype = str).fillna(0)

# check formats
checkInputs(sps,mt_biosyn,sp_types)

# layouts for destination plate(s)
layout_biosyn1_file = folder + 'biosyn_pl.csv' 
layout_biosyn1 = pd.read_csv(layout_biosyn1_file, index_col = 0, dtype = str)
# filename = '221018_arpae_biosyn_neg.csv'
# layout_biosyn2_file = 'plate_layouts/'+filename 
# layout_biosyn2 = pd.read_csv(layout_biosyn2_file, index_col = 0, dtype = str)
# layouts = [layout_biosyn1,layout_biosyn2]

vol_table_df = generateVolumeTable(mt_biosyn, sps, rxn_vol = 22.5, total_vol = 25, fill_with='HEPES')

protocol_biosyn_dfs = writeProtocol(sp_types, vol_table_df, layout_biosyn1,sps, update_source_vol= folder +'combined_sp_updated.xlsx')

#protocol_biosyn_df.to_csv('protocols/221018_arpae_biosyn.csv',index = False)

/var/folders/55/z2skprpx0m3_6k8fs3mcm_mm0000gn/T/ipykernel_73095/139932593.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sp_hepes = sp_hepes.append({'Label':'HEPES', 'Well':'A2,A3', 'Concentration':'', 'Volume':'2000,2000'}, ignore_index = True)
/Users/ryancardiff/Documents/GitHub/WHISPR/whispr.py:182: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sp = sp.append({'Label':label, 'Well':','.join(wells), 'Concentration':100, 'Volume':','.join([str(volumes)]*len(wells))}, ignore_index = True)
/Users/ryancardiff/Documents/GitHub/WHISPR/whispr.py:182: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sp = sp.append({'Label':label, 'Well':','.join(wells), 'Concentration':100, 'Volume':','.join([str(volumes)]*len(wells))}, ignor

TypeError: sort_values() got an unexpected keyword argument 'by'

In [57]:
protocol_biosyn_dfs[0].to_csv('protocols/221024_arpae_biosyn_384.csv',index = False)
protocol_biosyn_dfs[1].to_csv('protocols/221024_arpae_biosyn_6RES.csv',index = False)